# LLM agents with memory in neo4j


## Prepare the API credential

In [2]:
from autogen import ConversableAgent
from autogen import config_list_from_json

with open('../openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o-mini'

config_list = config_list_from_json(env_or_file="../CONFIG_LIST", filter_dict={"model": MODEL})
config_list[0]["api_key"] = key

### A cypher executor

In [ ]:
from tools.CypherExecutor import CypherCodeExecutor

cypher_executor = ConversableAgent(
    "cypher_executor",
    human_input_mode= "NEVER",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": CypherCodeExecutor()})

### A graph expert

In [ ]:
graph_expert = ConversableAgent(
    "graph_expert",
    system_message = """You are curious about person, and you can use a tool (linked to a graph database) to store the information about the persons you heard. Once you get a description about a person, find the person in the graph database as a node (or create one if it does not exist in the database), and save the description as well. After that, please identify all the other persons mentioned in this description, save them into the graph as nodes, together with the relation between the original person and the other person. Finally, find one person from the graph who does not have a description yet, and say 'I want to know more about <the name of this person>. TERMINATE'. Try not to query the database too often and to do so use cypher format
    Now i will give you some more information about the database schema:
    (:Person); contains a description and eventually him/her aliases
    (:Person)-[:ASSOCIATE]->(:Person); connect two person if they know each other
    (:Person)-[:CONTEMPORARY]->(:Person); connect two person if they lived in the same period""",
    llm_config = {"config_list": config_list},
    code_execution_config=False
)

### A schema dumper

In [ ]:
from autogen import GroupChat, GroupChatManager
from tools.tools import retrieve_content

#TODO: is there a way to have this as a generic function? in tools.tools
def state_transition(last_speaker, groupchat):
    if "cypher" in groupchat.messages[-1]["content"]: return cypher_executor
    elif last_speaker is person_expert: return graph_expert
    else: return person_expert


person_expert = ConversableAgent(
    "person_expert",
    system_message = "You are a helpful assistant. Once you were asked about a person, reply a short description (<100 words) about this person, including his/her relation with some other persons in the description.",
    llm_config = {"config_list": config_list},
    #is_termination_msg=lambda msg: "" in msg["content"].lower(),
    human_input_mode="NEVER",
)

groupchat = GroupChat(agents=[person_expert, graph_expert, cypher_executor], messages=[],
                          max_round=10, speaker_selection_method=state_transition)
    
manager   = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list})

PROBLEM = "I want to know something about Zhou Shuren"

In [ ]:
person_expert.initiate_chat(
    manager,
    message=PROBLEM,
    max_turns=2)

person_expert (to chat_manager):

I want to know something about Zhou Shuren

--------------------------------------------------------------------------------

Next speaker: graph_expert


>>>>>>>> USING AUTO REPLY...
graph_expert (to chat_manager):

Please provide me with a description of Zhou Shuren, and I will store it in the graph database.

--------------------------------------------------------------------------------

Next speaker: person_expert

person_expert (to chat_manager):

Zhou Shuren, better known by his pen name Lu Xun, was a prominent Chinese writer and intellectual of the early 20th century. He is regarded as one of the founding figures of modern Chinese literature. His works, including "A Madman's Diary" and "The True Story of Ah Q," critiqued societal norms and explored the struggles of the Chinese people during a time of great change. Lu Xun was also a mentor to many young writers, including Ding Ling, who admired his revolutionary spirit and literary style. He re

[]

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: The property existence syntax `... exists(variable.property)` is no longer supported. Please use `variable.property IS NOT NULL` instead. (line 3, column 11 (offset: 67))
"WHERE NOT EXISTS(p.description)"
           ^}

cypher_executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
{code: Neo.ClientError.Statement.SyntaxError} {message: The property existence syntax `... exists(variable.property)` is no longer supported. Please use `variable.property IS NOT NULL` instead. (line 3, column 11 (offset: 67))
"WHERE NOT EXISTS(p.description)"
           ^}

--------------------------------------------------------------------------------

Next speaker: person_expert

person_expert (to chat_manager):

It looks like the syntax for checking property existence has changed. You can use `IS NULL` instead. Here's the corrected query:

```cypher
// Find a person without a description
MATCH (p:Person) 
WHERE p.description IS NULL 
RETURN p.name LIMIT 1
```

You should be able to run this query successfully to find a person in the graph without a description.

--------------------------------------------------------------------------------

Next speaker: cypher_executor


>>>>>>>> EXECUTING CODE BLO

[{'p.name': 'Ding Ling'}]

...

[]

cypher_executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------

Next speaker: person_expert

[autogen.oai.client: 10-24 10:31:51] {315} WARNING - Model gpt-4o-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
person_expert (to chat_manager):

The description of Ding Ling has been successfully added to the graph database. If you have any other persons or topics you'd like to explore or any specific queries, feel free to ask!

--------------------------------------------------------------------------------

Next speaker: graph_expert


>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 10-24 10:31:55] {315} WARNING - Model gpt-4o-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pri

[]

cypher_executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'I want to know something about Zhou Shuren', 'role': 'assistant'}, {'content': 'Please provide me with a description of Zhou Shuren, and I will store it in the graph database.', 'name': 'graph_expert', 'role': 'user'}, {'content': 'Zhou Shuren, better known by his pen name Lu Xun, was a prominent Chinese writer and intellectual of the early 20th century. He is regarded as one of the founding figures of modern Chinese literature. His works, including "A Madman\'s Diary" and "The True Story of Ah Q," critiqued societal norms and explored the struggles of the Chinese people during a time of great change. Lu Xun was also a mentor to many young writers, including Ding Ling, who admired his revolutionary spirit and literary style. He remains a significant cultural icon in China.', 'role': 'assistant'}, {'content': 'I\'ll store the description of Zhou Shuren (Lu Xun) in the graph database, and then I\'ll add Ding Ling as a related person. \n